# Transform_indents

A partir da função _transform_legislations o texto será retornado com os tokens de recuo criados.

Essa função padroniza as chamadas de alíneas, de forma que:
- alinea A seja substituído por ST_ALINEA_A




In [1]:
import re

ST_PREFIX = "ST_"

def _transform_legislations(
    text,
    prefix_result,
    re_legislation,
    is_multiple=False,
    identification=None,
    previous_token=None,
    separator=None,
):
    "Create custom tokens for legislation"

    def build_token(m):
        if m.group("ignored_expression"):
            result = m.group("ignored_expression")
        else:
            result = ""
        result = (
            result
            + ST_PREFIX
            + prefix_result
            + m.group("identification")
            .replace('"', "")
            .replace("“", "")
            .replace("”", "")
            .replace(".", "")
            .replace(",", "")
            .replace("º", "")
            .lstrip("0")
            .upper()
        )
        return result

    def build_token_multi(m):
        result = (
            m.group("ignored_expression")
            + ST_PREFIX
            + prefix_result
            + m.group("identification")
            .replace('"', "")
            .replace("“", "")
            .replace("”", "")
            .replace(".", "")
            .replace(",", "")
            .replace("º", "")
            .lstrip("0")
            .upper()
        )
        return result

    # replaces first occurrences
    text = re_legislation.sub(build_token, text)

    # replaces multiple occurrences, if exists
    if is_multiple:
        regex_legislation_multi = re.compile(
            fr"(?:(?P<ignored_expression>{previous_token}{separator}){identification})",
            flags=re.I,
        )
        while regex_legislation_multi.findall(text) != []:
            text = regex_legislation_multi.sub(build_token_multi, text)

    return text


In [2]:
def transform_indents(text, **kwargs):
    #Create custom tokens for indents

    prefix_result = "ALINEA_"
    previous_token = fr"{ST_PREFIX}{prefix_result}\w+" #ST_ALINEA_
    separator = r"\s*[,e]\s+"
    indent = (
        r"(?<!\w)[\"|\“|\”]?(?P<identification>[a|b]?[a-z])[\"|\“|\”]?(?!\w)"
    )
    re_legislation = re.compile(
        fr"(?P<ignored_expression>)(?:al[í|i]neas?\s+{indent})", flags=re.I
    )

    return _transform_legislations(
        text,
        prefix_result,
        re_legislation,
        True,
        indent,
        previous_token,
        separator,
    )

In [3]:
entradas = """
    fundamento nas alíneas “a”, “ac” e "bc", do inciso III, do art. 102
    art. 102, inciso III, alinea “a” da Constituição Federal
"""

print("Entradas: ", entradas)

Entradas:  
    fundamento nas alíneas “a”, “ac” e "bc", do inciso III, do art. 102
    art. 102, inciso III, alinea “a” da Constituição Federal



In [4]:
print("Saida: ",transform_indents(entradas))

Saida:  
    fundamento nas ST_ALINEA_A, ST_ALINEA_AC e ST_ALINEA_BC, do inciso III, do art. 102
    art. 102, inciso III, ST_ALINEA_A da Constituição Federal



# DATASET VICTOR

In [5]:
import pandas as pd

test = pd.read_csv('./data/train_small.csv')
text = test['body']

In [6]:
print("Dados de entrada:\n",text)

Dados de entrada:
 0         {"tribunal justiça estado bahia poder judiciár...
1         {"excelentíssimo senhor doutor juiz direito ju...
2         {"razões recurso inominado recorrente atlantic...
3         {"empresa recorrente tornou credora dos débito...
4         {"entretanto verdade parte apelante tornou tit...
                                ...                        
149212    {"supremo tribunal federal recurso extraordiná...
149213    {"seção recursos extraordinários mandado intim...
149214    {"ttar qsvòwi a edewrr seção recursos extraord...
149215    {"ertidao certifico dou que dirigi setor autar...
149216    {"supremo tribunal federal secretaria judiciár...
Name: body, Length: 149217, dtype: object


In [13]:
lista = []
for item in text:
    if "alíneas" in item:
        lista.append(transform_indents(item))

In [15]:
print("Saida: \n", lista[6])

Saida: 
 {"advocacia geral união procuradoria geral fazenda nacional procuradoria fazenda nacional alagoas mesmo sentido ARTIGO_29 caput LEI_8213 ARTIGO_29 salário benefício consiste para benefícios que tratam ST_ALINEA_E do inciso do ARTIGO_18 média aritmética simples dos maiores salários contribuição correspondentes oitenta por cento todo período contributivo multiplicada pelo fator previdenciário incluído pela LEI_9876 para benefícios que tratam ST_ALINEA_H inciso do ARTIGO_18 média aritmética simples dos maiores salários decontribuição correspondentes oitenta por cento todo período contributivo incluído pela LEI_9876 serão considerados para cálculo salário benefício ganhos habituais segurado empregado qualquer título sob forma moeda corrente utilidades sobre quais tenha incidido contribuições previdenciárias exceto décimo terceiro salário gratificação natalina redação dada pela LEI_8870 espécie própria lei que instituiu gacen estendeu aos futuros benefícios previdenciários dos cont